In [17]:
import pandas as pd
import nibabel as nb
import os
import shutil

In [4]:
def organise_target_and_volumeid(df_path='/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/10_1573078374.453554_report.csv'):
    df = pd.read_csv(df_path, index_col=0)
    if df['dataset'].values[1] != 2:
        print('Not a UKB dataset report, not processing!')
        return False
    def replace_func(value):
        string = value[-1]
        return f'Dixon_BH_17s_opp_Dixon_BH_17{string}.nii.gz' if string == 's' else f'Dixon_BH_17s_opp_Dixon_BH_17s{string}.nii.gz'
    df['target_scan_file'] = df['volume_id'].apply(replace_func)
    df['volume_id'] = df['volume_id'].apply(lambda x: x[:-1])
    return df

def concat_partial_model_reports(paths):
    df = None
#     ids = ['10_1573078374.453554', '10_1573225388.879571', '10_1573391294.0298784']
#     paths = [f'/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/{i}_report.csv' for i in ids]
    
    for ix, p in enumerate(paths):
        df_ = organise_target_and_volumeid(p)
        df_ = df_.drop(df_.index[0])
        if ix == 0:
            df = df_
        else:
            df = df.append(df_, ignore_index=True)
    return df

In [5]:
ids = ['10_1573078374.453554', '10_1573225388.879571', '10_1573391294.0298784']
paths = [f'/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/{i}_report.csv' for i in ids]
df = concat_partial_model_reports(paths)  

In [8]:
df.to_csv('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/10_0.0_report.csv')

In [9]:
df = pd.read_csv('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/10_0.0_report.csv', index_col=0)

In [18]:
df

,dataset,model_name,volume_id,samples,sncc,ged,iou_spleen,iou_liver,iou_mean,dice_spleen,...,surface_distance_prtogt_spleen,surface_distance_prtogt_liver,surface_distance_prtogt_mean,surface_distance_gttopr_spleen,surface_distance_gttopr_liver,surface_distance_gttopr_mean,surface_distance_avg_spleen,surface_distance_avg_liver,surface_distance_avg_mean,target_scan_file
0,2,MC_dropout_quicknat_UKB_v2,1000410_20201_2_0,10,NaN,NaN,0.545999,0.859389,0.702694,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz
1,2,MC_dropout_quicknat_UKB_v2,1000698_20201_2_0,10,NaN,NaN,0.564894,0.766553,0.665723,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz
2,2,MC_dropout_quicknat_UKB_v2,1000857_20201_2_0,10,NaN,NaN,0.719200,0.851086,0.785143,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz
3,2,MC_dropout_quicknat_UKB_v2,1000870_20201_2_0,10,NaN,NaN,0.738249,0.585848,0.662049,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz
4,2,MC_dropout_quicknat_UKB_v2,1000870_20201_2_0,10,NaN,NaN,0.335961,0.356094,0.346028,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz
5,2,MC_dropout_quicknat_UKB_v2,1002272_20201_2_0,10,NaN,NaN,0.436899,0.369021,0.402960,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz
6,2,MC_dropout_quicknat_UKB_v2,1002272_20201_2_0,10,NaN,NaN,0.875844,0.705661,0.790752,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz
7,2,MC_dropout_quicknat_UKB_v2,1002359_20201_2_0,10,NaN,NaN,0.849520,0.884133,0.866826,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz
8,2,MC_dropout_quicknat_UKB_v2,1002876_20201_2_0,10,NaN,NaN,0.776984,0.819199,0.798091,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz
9,2,MC_dropout_quicknat_UKB_v2,1003082_20201_2_0,10,NaN,NaN,0.418048,0.798031,0.608040,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz


In [12]:
with open('/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/UKB/test_volumes.txt') as file_handle:
        volumes_to_use = file_handle.read().splitlines()

In [14]:
len(volumes_to_use)

11139

In [ ]:

for ix, v in enumerate(volumes_to_use):
    print("\rProcessing {}/{}.".format(ix, len(volumes_to_use)), end="")
    targets = df[df['volume_id']==v]['target_scan_file'].values
    i1, i2 = None, None
    if len(targets)>1:
        i1, i2 = df[df['volume_id']==v]['iou_mean'].values
        if i1>i2:
            idx = 0
        else:
            idx = 1
        target = targets[idx]
    elif len(targets)==0:
        continue
    else:
        target = targets[0]

#     input_file = '/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/'+str(v)+'/'+target
#     gt_file = None
#     f_paths.append([input_file, gt_file])
    postfix = target.split('.')[0][-1]
    shutil.copy2(f'/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_processed/data/{v}{postfix}.nii.gz',
                 f'/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_processed/data_/{v}.nii.gz')


Processing 248/11139.

In [ ]:
len(f_paths)

In [ ]:
f_paths

In [ ]:
import torch

In [ ]:
model = torch.load('/home/abhijit/Jyotirmay/my_thesis/projects/full_bayesian/saved_models/full_bayesian_KORA_v2.pth.tar')

In [ ]:
torch.save(model.state_dict(), '/home/abhijit/Jyotirmay/whole_body_pipeline/nn_api/nn_api/pytorch_models/full_bayesian.model')